In [3]:
import os
import pandas as pd

def encontrar_raiz_proyecto(archivo_raiz="README.md") -> str:
    """Encuentra la raíz del proyecto buscando un archivo identificador"""
    ruta = os.path.abspath(os.getcwd())
    while True:
        if archivo_raiz in os.listdir(ruta):
            return ruta
        nueva_ruta = os.path.dirname(ruta)
        if nueva_ruta == ruta:  # Llegamos a la raíz del sistema sin encontrar
            raise FileNotFoundError(f"No se encontró {archivo_raiz} en ningún directorio padre")
        ruta = nueva_ruta

# Obtiene la ruta absoluta a la raíz del proyecto y la carpeta data
DIR_RAIZ = encontrar_raiz_proyecto()
DIR_DATA = os.path.join(DIR_RAIZ, "data")

# Rutas a los archivos
RUTA_TITULO = os.path.join(DIR_DATA, "title.basics.tsv")
RUTA_NOMBRE = os.path.join(DIR_DATA, "name.basics.tsv")
RUTA_PERSONAL = os.path.join(DIR_DATA, "title.crew.tsv")
RUTA_EPISODIO = os.path.join(DIR_DATA, "title.episode.tsv")
RUTA_PRINCIPALES = os.path.join(DIR_DATA, "title.principals.tsv")
RUTA_CRITICAS = os.path.join(DIR_DATA, "title.ratings.tsv")

def leer_tsv_chunks(ruta: str):
    return pd.read_csv(ruta, sep='\t', chunksize=10000)

In [10]:
titulos_iterable = leer_tsv_chunks(RUTA_TITULO)

# Conjunto para almacenar tipos únicos
tipos_unicos = set()

# Leer chunk por chunk y añadir los tipos únicos al conjunto
for chunk in titulos_iterable:
    tipos_unicos.update(chunk['titleType'].unique())

# Convertir a lista y ordenar alfabéticamente
lista_tipos = sorted(list(tipos_unicos))
print("Tipos de títulos únicos encontrados:")
for tipo in lista_tipos:
    print(f"- {tipo}")

Tipos de títulos únicos encontrados:
- movie
- short
- tvEpisode
- tvMiniSeries
- tvMovie
- tvPilot
- tvSeries
- tvShort
- tvSpecial
- video
- videoGame


In [11]:
# Lista de tipos que queremos filtrar
tipos_buscados = ['short', 'tvMovie', 'tvShort', 'tvSpecial']

# Diccionario para almacenar los resultados de cada tipo
resultados = {tipo: [] for tipo in tipos_buscados}

# Leer chunks y filtrar
titulos_iterable = leer_tsv_chunks(RUTA_TITULO)

for chunk in titulos_iterable:
    for tipo in tipos_buscados:
        # Si aún no tenemos 5 registros de este tipo, buscamos más
        if len(resultados[tipo]) < 5:
            filtrados = chunk[chunk['titleType'] == tipo]
            # Tomamos solo los que faltan hasta llegar a 5
            resultados[tipo].extend(filtrados.head(5 - len(resultados[tipo])).to_dict('records'))
    
    # Verificar si ya tenemos 5 de cada tipo
    if all(len(registros) >= 5 for registros in resultados.values()):
        break

# Mostrar resultados
for tipo in tipos_buscados:
    print(f"\n=== {tipo.upper()} ===")
    print("Título Original | Año Inicio | Duración | Géneros")
    print("-" * 70)
    for registro in resultados[tipo]:
        print(f"{registro['originalTitle'][:30]:<30} | {registro['startYear']:<10} | {registro['runtimeMinutes']:<8} | {registro['genres']}")


=== SHORT ===
Título Original | Año Inicio | Duración | Géneros
----------------------------------------------------------------------
Carmencita                     | 1894       | 1        | Documentary,Short
Le clown et ses chiens         | 1892       | 5        | Animation,Short
Pauvre Pierrot                 | 1892       | 5        | Animation,Comedy,Romance
Un bon bock                    | 1892       | 12       | Animation,Short
Blacksmith Scene               | 1893       | 1        | Short

=== TVMOVIE ===
Título Original | Año Inicio | Duración | Géneros
----------------------------------------------------------------------
Julius Caesar                  | 1938       | 101      | Drama,History
As You Like It                 | 1946       | \N       | Drama
A Midsummer Night's Dream      | 1946       | 150      | Drama,Fantasy
Hamlet Part 1                  | 1947       | 88       | Drama
The Merchant of Venice         | 1947       | 90       | Drama

=== TVSHORT ===
Título Origi

In [12]:
# Tipos de películas que queremos mantener
tipos_peliculas = ['movie', 'tvMovie']

# Lista para ir acumulando los chunks filtrados
peliculas_filtradas = []

# Leer y filtrar por chunks
titulos_iterable = leer_tsv_chunks(RUTA_TITULO)
for chunk in titulos_iterable:
    # Filtrar solo los tipos que queremos
    chunk_filtrado = chunk[chunk['titleType'].isin(tipos_peliculas)]
    peliculas_filtradas.append(chunk_filtrado)

# Concatenar todos los chunks filtrados en un único DataFrame
df_peliculas = pd.concat(peliculas_filtradas, ignore_index=True)

# Mostrar información general
print(f"Total de registros filtrados: {len(df_peliculas)}")
print("\nDistribución por tipo:")
print(df_peliculas['titleType'].value_counts())

print("\nPrimeras 5 películas de cada tipo:")
for tipo in tipos_peliculas:
    print(f"\n=== {tipo.upper()} ===")
    print(df_peliculas[df_peliculas['titleType'] == tipo].head()[['primaryTitle', 'startYear', 'runtimeMinutes', 'genres']])

Total de registros filtrados: 876,594

Distribución por tipo:
titleType
movie      724491
tvMovie    152103
Name: count, dtype: int64

Primeras 5 películas de cada tipo:

=== MOVIE ===
                    primaryTitle startYear runtimeMinutes  \
0                     Miss Jerry      1894             45   
1  The Corbett-Fitzsimmons Fight      1897            100   
2                       Bohemios      1905            100   
3    The Story of the Kelly Gang      1906             70   
4               The Prodigal Son      1907             90   

                       genres  
0                     Romance  
1      Documentary,News,Sport  
2                          \N  
3  Action,Adventure,Biography  
4                       Drama  

=== TVMOVIE ===
                    primaryTitle startYear runtimeMinutes         genres
20302              Julius Caesar      1938            101  Drama,History
26711             As You Like It      1946             \N          Drama
27062  A Midsummer N

In [13]:
# Función para contar valores nulos y '\N'
def contar_nulos(df, columna):
    nulos_N = df[df[columna] == '\\N'].shape[0]  # Contar '\N'
    nulos_nan = df[columna].isna().sum()         # Contar NaN
    total = nulos_N + nulos_nan
    porcentaje = (total / len(df)) * 100
    return {
        'Valores \\N': nulos_N,
        'Valores NaN': nulos_nan,
        'Total Nulos': total,
        'Porcentaje': f"{porcentaje:.2f}%"
    }

# Analizar cada columna
print("Análisis de valores nulos por columna:")
print("-" * 50)

for columna in df_peliculas.columns:
    resultados = contar_nulos(df_peliculas, columna)
    print(f"\nColumna: {columna}")
    print(f"- Valores '\\N': {resultados['Valores \\N']:,}")
    print(f"- Valores NaN: {resultados['Valores NaN']:,}")
    print(f"- Total nulos: {resultados['Total Nulos']:,}")
    print(f"- Porcentaje: {resultados['Porcentaje']}")

# Mostrar también cuántos registros tienen al menos un valor nulo
registros_con_nulos = df_peliculas[df_peliculas.apply(lambda x: (x == '\\N').any() or x.isna().any(), axis=1)]
print(f"\nRegistros con al menos un valor nulo: {len(registros_con_nulos):,} ({(len(registros_con_nulos)/len(df_peliculas)*100):.2f}% del total)")

Análisis de valores nulos por columna:
--------------------------------------------------

Columna: tconst
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: titleType
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: primaryTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: originalTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: isAdult
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: startYear
- Valores '\N': 111,390
- Valores NaN: 0
- Total nulos: 111,390
- Porcentaje: 12.71%

Columna: endYear
- Valores '\N': 876,594
- Valores NaN: 0
- Total nulos: 876,594
- Porcentaje: 100.00%

Columna: runtimeMinutes
- Valores '\N': 314,612
- Valores NaN: 0
- Total nulos: 314,612
- Porcentaje: 35.89%

Columna: genres
- Valores '\N': 90,763
- Valores NaN: 0
- Total nulos: 90,763
- Porcentaje: 10.35%

Registros con al menos un va

In [14]:
# Columnas a verificar
columnas_a_filtrar = ['genres', 'primaryTitle', 'originalTitle', 'runtimeMinutes']

# Crear una máscara para filtrar
mascara = True  # Comenzamos con todos los registros
for columna in columnas_a_filtrar:
    # Filtrar tanto '\N' como NaN
    mascara = mascara & (df_peliculas[columna] != '\\N') & (~df_peliculas[columna].isna())

# Aplicar el filtro
df_peliculas_limpio = df_peliculas[mascara]

# Mostrar información sobre los registros filtrados
registros_eliminados = len(df_peliculas) - len(df_peliculas_limpio)
print(f"Registros originales: {len(df_peliculas):,}")
print(f"Registros después de eliminar nulos: {len(df_peliculas_limpio):,}")
print(f"Registros eliminados: {registros_eliminados:,} ({(registros_eliminados/len(df_peliculas)*100):.2f}%)")

print("\nDistribución por tipo después de la limpieza:")
print(df_peliculas_limpio['titleType'].value_counts())

print("\nMuestra de 5 registros limpios:")
print(df_peliculas_limpio[['primaryTitle', 'titleType', 'startYear', 'runtimeMinutes', 'genres']].head())

Registros originales: 876,594
Registros después de eliminar nulos: 530,367
Registros eliminados: 346,227 (39.50%)

Distribución por tipo después de la limpieza:
titleType
movie      431138
tvMovie     99229
Name: count, dtype: int64

Muestra de 5 registros limpios:
                     primaryTitle titleType startYear runtimeMinutes  \
0                      Miss Jerry     movie      1894             45   
1   The Corbett-Fitzsimmons Fight     movie      1897            100   
3     The Story of the Kelly Gang     movie      1906             70   
4                The Prodigal Son     movie      1907             90   
8  The Fairylogue and Radio-Plays     movie      1908            120   

                       genres  
0                     Romance  
1      Documentary,News,Sport  
3  Action,Adventure,Biography  
4                       Drama  
8           Adventure,Fantasy  


In [22]:
# Filtrar películas del 2019
df_peliculas_anio = df_peliculas_limpio[df_peliculas_limpio['startYear'] == '2019']

print(f"Total de películas del 2019: {len(df_peliculas_anio):,}")
print("\nDistribución por tipo:")
print(df_peliculas_anio['titleType'].value_counts())

print("\nEstadísticas de duración (en minutos):")
print(df_peliculas_anio['runtimeMinutes'].astype(float).describe())

print("\nMuestra de películas del 2019:")
muestra = df_peliculas_anio[['primaryTitle', 'titleType', 'runtimeMinutes', 'genres']].head(10)
print(muestra.to_string())

Total de películas del 2019: 16,833

Distribución por tipo:
titleType
movie      14637
tvMovie     2196
Name: count, dtype: int64

Estadísticas de duración (en minutos):
count    16833.000000
mean        93.571021
std        514.616846
min          1.000000
25%         70.000000
50%         88.000000
75%        101.000000
max      59460.000000
Name: runtimeMinutes, dtype: float64

Muestra de películas del 2019:
                 primaryTitle titleType runtimeMinutes                   genres
87763     Mariette in Ecstasy     movie            101                    Drama
217353    Motherless Brooklyn     movie            144      Crime,Drama,Mystery
233764    Alita: Battle Angel     movie            122  Action,Adventure,Sci-Fi
235270           Danger Close     movie            118         Action,Drama,War
237327                Shazam!     movie            132  Action,Adventure,Comedy
254146  The Last Full Measure     movie            116                Drama,War
254769                Ene

In [5]:
nombre_iterable = leer_tsv_chunks(RUTA_NOMBRE)

ultimo_chunk = None
for chunk in nombre_iterable:
    ultimo_chunk = chunk
    
ultimo_chunk.tail(10)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
14677622,nm9993709,Lu Bevins,\N,\N,"producer,writer,director","tt17717854,tt11772904,tt11772812,tt11697102"
14677623,nm9993710,Nestor Rudnytskyy,\N,\N,\N,\N
14677624,nm9993711,David Gluzman,\N,\N,\N,\N
14677625,nm9993712,Corny O'Connell,\N,\N,\N,\N
14677626,nm9993713,Sambit Mishra,\N,\N,"writer,producer","tt20319332,tt27191658,tt10709066,tt15134202"
14677627,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
14677628,nm9993716,Essias Loberg,\N,\N,\N,\N
14677629,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
14677630,nm9993718,Aayush Nair,\N,\N,cinematographer,tt8736744
14677631,nm9993719,Andre Hill,\N,\N,\N,\N


In [6]:
personal_iterable = leer_tsv_chunks(RUTA_PERSONAL)

ultimo_chunk = None
for chunk in personal_iterable:
    ultimo_chunk = chunk
    
ultimo_chunk.tail(10)

,tconst,directors,writers
11875181,tt9916838,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
11875182,tt9916840,nm0996406,"nm1482639,nm2586970"
11875183,tt9916842,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
11875184,tt9916844,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
11875185,tt9916846,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
11875186,tt9916848,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
11875187,tt9916850,nm1485677,"nm9187127,nm1485677,nm9826385,nm1628284"
11875188,tt9916852,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
11875189,tt9916856,nm10538645,nm6951431
11875190,tt9916880,nm0996406,"nm1482639,nm2586970"


In [7]:
episodio_iterable = leer_tsv_chunks(RUTA_EPISODIO)

ultimo_chunk = None
for chunk in episodio_iterable:
    ultimo_chunk = chunk
    
ultimo_chunk.tail(10)

,tconst,parentTconst,seasonNumber,episodeNumber
9139769,tt9916836,tt1289683,3,14
9139770,tt9916838,tt1289683,3,13
9139771,tt9916840,tt0985991,4,1
9139772,tt9916842,tt1289683,3,16
9139773,tt9916844,tt1289683,3,15
9139774,tt9916846,tt1289683,3,18
9139775,tt9916848,tt1289683,3,17
9139776,tt9916850,tt1289683,3,19
9139777,tt9916852,tt1289683,3,20
9139778,tt9916880,tt0985991,4,2


In [8]:
principales_iterable = leer_tsv_chunks(RUTA_PRINCIPALES)

ultimo_chunk = None
for chunk in principales_iterable:
    ultimo_chunk = chunk
    
ultimo_chunk.tail(10)

,tconst,ordering,nconst,category,job,characters
94444251,tt9916880,12,nm2676923,actress,\N,"[""Sour Susan""]"
94444252,tt9916880,13,nm2676923,actress,\N,"[""Goody-Goody Gordon""]"
94444253,tt9916880,14,nm2676923,actress,\N,"[""Singing Soraya""]"
94444254,tt9916880,15,nm1469295,actress,\N,"[""Perfect Peter""]"
94444255,tt9916880,16,nm1469295,actress,\N,"[""Lazy Linda""]"
94444256,tt9916880,17,nm0996406,director,principal director,\N
94444257,tt9916880,18,nm1482639,writer,\N,\N
94444258,tt9916880,19,nm2586970,writer,books,\N
94444259,tt9916880,20,nm1594058,producer,producer,\N
94444260,tt9916880,21,nm1482639,producer,producer,\N


In [9]:
criticas_iterable = leer_tsv_chunks(RUTA_CRITICAS)

ultimo_chunk = None
for chunk in criticas_iterable:
    ultimo_chunk = chunk
    
ultimo_chunk.tail(10)

,tconst,averageRating,numVotes
1607719,tt9916836,5.8,6
1607720,tt9916838,5.8,6
1607721,tt9916840,6.8,12
1607722,tt9916842,5.6,6
1607723,tt9916844,5.6,6
1607724,tt9916846,5.8,6
1607725,tt9916848,5.8,6
1607726,tt9916850,5.8,6
1607727,tt9916852,5.8,6
1607728,tt9916880,7.7,10


In [ ]:
def direccionArchivo():
    absFilePath=os.path.abspath(__file__)
    path,filename=os.path.split(absFilePath)

    return path

def crearData():
    path=direccionArchivo()
    return pd.read_csv(path+"./mundial.csv")